In [9]:
import requests
from bs4 import BeautifulSoup
import time
from openpyxl import Workbook, load_workbook
import os

file_name = f"04_일반뉴스_암_기사_20250326~20250425.xlsx"
keyword = '크롤링'
lastpage = 3  # 원하시는 페이지 수 입력

if os.path.exists(file_name):
    wb = load_workbook(file_name)
    ws = wb.active
else:
    wb = Workbook()
    ws = wb.active
    ws.title = '일반뉴스기사'
    ws.append(['제목', '날짜', 'URL', '내용'])

ws.column_dimensions['A'].width = 60
ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 60
ws.column_dimensions['D'].width = 150

pageNum = 1
for i in range(1, lastpage * 10, 10):
    print(f"{pageNum}페이지 크롤링중입니다 =================")

    url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2025.04.14&de=2025.04.20&start={i}"
    response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = soup.select("div.news_wrap.api_ani_send")

    for article in articles:
        link_tag = article.select_one("a.news_tit")
        if not link_tag:
            continue

        url = link_tag.get('href')
        title = link_tag.get('title')

        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        soup_sub = BeautifulSoup(response.text, 'html.parser')

        date = '날짜 없음'
        time_tags = soup_sub.find_all('time')
        for tag in time_tags:
            if '발행일' in tag.text:
                date = tag.text.replace('발행일 :', '').strip()
                break

        content = '내용 없음'
        article_body = soup_sub.find('div', id='articleBody')
        if article_body:
            paragraphs = article_body.find_all('p')
            content = '\n\n'.join(p.get_text(separator=' ', strip=True) for p in paragraphs)

        ws.append([title or '제목 없음', date, url, content])
        time.sleep(0.6)

    pageNum += 1

wb.save(file_name)
print(f"엑셀 저장 완료: {file_name}")


1페이지 크롤링중입니다 =================
2페이지 크롤링중입니다 =================
3페이지 크롤링중입니다 =================
엑셀 저장 완료: 04_일반뉴스_암_기사_20250326~20250425.xlsx
